In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# 🔹 환경변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print("🔑 OpenAI API Key:", OPENAI_API_KEY)
# 🔹 PDF → 벡터스토어 저장 함수
def load_and_store_pdf(file_path, persist_dir="vector_store"):
    loader = PyPDFLoader(file_path)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)  # ⬅️ Token 문제 방지
    split_docs = splitter.split_documents(docs)

    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")
    vectordb = Chroma.from_documents(split_docs, embedding, persist_directory=persist_dir)
    vectordb.persist()
    print("✅ 벡터스토어 저장 완료")
    return vectordb

# 🔹 RAG QA 체인 반환 함수
def get_rag_chain(persist_dir="vector_store"):
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")
    vectordb = Chroma(persist_directory=persist_dir, embedding_function=embedding)
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})  # 검색 개수 제한

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2, openai_api_key=OPENAI_API_KEY)
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=False  # ⬅️ token 초과 방지
    )
    return chain

# 🔹 질의응답 비교 함수
def compare_llm_vs_rag(query, rag_chain):
    # 기본 LLM 응답 (문서 검색 안 쓰고 순수 GPT만)
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2, openai_api_key=OPENAI_API_KEY)
    llm_response = llm.predict(query)

    # RAG 응답 (문서 기반 검색)
    rag_response = rag_chain.run(query)

    return {
        "LLM 응답 (기본)": llm_response,
        "RAG 응답 (문서 기반)": rag_response
    }

# 🔹 테스트 예제 실행
if __name__ == "__main__":
    pdf_path = "KB 실버암 간편건강보험Plus.pdf"

    # 최초 실행 시: 벡터스토어 저장
    load_and_store_pdf(pdf_path)

    # RAG 체인 생성
    rag = get_rag_chain()

    # 예제 질의
    question = "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?"
    result = compare_llm_vs_rag(question, rag)

    # 결과 출력
    for k, v in result.items():
        print(f"\n🔸 {k}:\n{v}")

C:\Users\KDT4\AppData\Local\Temp\ipykernel_17088\2632734936.py:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")
C:\Users\KDT4\AppData\Local\Temp\ipykernel_17088\2632734936.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


✅ 벡터스토어 저장 완료


C:\Users\KDT4\AppData\Local\Temp\ipykernel_17088\2632734936.py:31: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_dir, embedding_function=embedding)
C:\Users\KDT4\AppData\Local\Temp\ipykernel_17088\2632734936.py:34: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2, openai_api_key=OPENAI_API_KEY)
C:\Us


🔸 LLM 응답 (기본):
갑상선암은 다음과 같은 검사를 통해 진단받을 수 있습니다:

1. 혈액검사: 갑상선 호르몬 수치와 갑상선 관련 마커인 TSH(TSH) 수치를 측정하여 갑상선 기능 이상을 확인할 수 있습니다.

2. 초음파 검사: 초음파를 이용하여 갑상선 종양의 크기, 형태, 위치 등을 확인할 수 있습니다.

3. 갑상선 바늘 생검: 갑상선 종양의 성질을 확인하기 위해 조직검사를 위해 바늘을 이용하여 채취한 조직을 검사할 수 있습니다.

의사의 판단에 따라 위 검사들 중 하나 또는 여러 가지를 시행하여 갑상선암을 진단받을 수 있습니다.

🔸 RAG 응답 (문서 기반):
"갑상선암"의 진단은 조직(fixed tissue)검사 또는 미세바늘흡인검사(fine needle aspiration) 또는 혈액(hemic system)검사에 대한 현미경 소견을 기초로 내려져야 합니다. 이러한 진단은 병리 또는 진단검사의학의 전문가에 의해 이루어져야 합니다.
